In [7]:
!pip install gradio transformers

In [8]:
import pandas as pd
import ast
import random

df = pd.read_csv("medical_knowledge_base.csv", encoding="ISO-8859-1", on_bad_lines='skip')

symptom_index = {}
for _, row in df.iterrows():
    try:
        symptoms = ast.literal_eval(row["symptom"])
        for symptom in symptoms:
            key = symptom.lower().strip()
            if key not in symptom_index:
                symptom_index[key] = []
            symptom_index[key].append({
                "name": row["name"],
                "desc": row["desc"],
                "cause": row["cause"],
                "prevent": row["prevent"],
                "cure_way": row["cure_way"]
            })
    except:
        continue


In [9]:
from transformers import pipeline


qa = pipeline("text-generation", model="gpt2")

Device set to use cpu


In [20]:
def shorten(text, max_words=40):
    words = text.split()
    return ' '.join(words[:max_words]) + '...' if len(words) > max_words else text

def medbot_chat(message, history):
    user_input = message.lower().strip()

    greetings = {
        "hi": "👋 Hello! I'm SwasthyaMitra. Tell me your symptoms.",
        "hello": "👋 Hi there! How can I help you today?",
        "hey": "👋 Hey! I'm your health buddy. What are you feeling?",
    }
    if user_input in greetings:
        return greetings[user_input]

    matched_conditions = []
    for word in user_input.split():
        if word in symptom_index:
            matched_conditions.extend(symptom_index[word])

    if matched_conditions:
        condition = random.choice(matched_conditions)
        desc = shorten(condition["desc"], 40)
        cause = shorten(condition["cause"], 30)
        prevent = shorten(condition["prevent"], 25)
        treatment = ', '.join(condition["cure_way"]) if isinstance(condition["cure_way"], list) else condition["cure_way"]

        return f"""🩺 Predicted Disease: {condition['name']}
📄 Description: {desc}
⚠️ Causes: {cause}
🛡️ Preventions: {prevent}
💊 Treatments: {treatment}
"""

    fallback = qa(user_input, max_length=60, num_return_sequences=1)[0]['generated_text']
    return "🤖 " + fallback


In [21]:
import gradio as gr

chat = gr.ChatInterface(
    fn=medbot_chat,
    title="🩺 SwasthyaMitra – AI Health Assistant",
    description="Chat with SwasthyaMitra. Enter your symptoms or just say hello!",
    theme="soft",
    submit_btn="Send",
)

chat.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://79c1cca9eacb3b2c90.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
